<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastian Avendaño
- Nombre de alumno 2: Felipe Urrutia


### **Link de repositorio de GitHub:** https://github.com/furrutiav/lab-mds-2022

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [3]:
#%%file bench_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [4]:
votes, post, url = praw_reddit()

### Estudiar la función propuesta por el equipo docente.

La funcion $\texttt{praw_reddit}$ recibe un subreddit (por defecto "chile") y un numero denotado por $\texttt{n_hot}$ (por defecto 1000). Retorna un resumen de los reslutados entregados por la API *praw* de los top-post mas "hot" en el subreddit establecido en el input. En particular, entre los votos (up/down votes), post (titulo del post) y url (una ruta a cada post). Internamente la funtion determina los votos segun dos parametros de cada post (o submission). Esto son: upvote_ratio y score. El primero corresponde a la proporcion de votos up del total de votos. Mientras que el segundo es la diferencia entre los votos up y los votos down. Para determinar los votos up, basta determinar (upvote_ratio * score) / (2 * upvote_ratio - 1). Luego, los votos down, se determinan con votos up - score. El output final de la funcion es un trio (votes, post, url), donde cada elemento del trio es una lista de listas. Donde cada sublista es una version truncada de la lista que le sigue. Esto ya que internamente la funcion realiza un ciclo for con pasos de tamaño 50, y almacena los datos de los primeros k hot-post, con k multiplo positivo de 50. A priori, consideramos que esta ultima observacion produce que el algoritmo no sea optimo, al realizar iteraciones innecesarias.

### Estudiar los tiempos de ejecución del código a través de un perfilador.

In [5]:
!pip install line_profiler
%load_ext line_profiler

In [6]:
%lprun -f praw_reddit praw_reddit()

<img src="perfilador.jpg" width=800 />

**Respuesta:**

Del perfilador se concluye que la mayoria del tiempo (99.7% del tiempo de ejecucion de la funcion) es consumida por la linea 13. Esto corresponde al llamado subreddit.hot(limit=n_hot) con el objeto "subreddit" y metodo "hot". Este resultado es esperable dado que en la linea 2 se inicializa el wrapper (de la libreria praw) con las credenciales, lo que no consume tiempo. Pero, en la linea 13 se realiza la primera peticion a la API. Ademas, este tiempo puede ser grande ya que el parametro 𝚗⎯𝚑𝚘𝚝 tambien lo es. Ahora bien, si omitimos el tiempo que consume la linea 13, el otro gran porcentaje de tiempo utilizado se debe al ciclo for que comienza en la linea 18.

### Estudiar la memoria ocupada por el código a través de un perfilador.

In [7]:
!pip install memory_profiler
%load_ext memory_profiler

In [8]:
from bench_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

<img src="perfilador_mem.jpg" width=800 />

**Respuesta:**

Del perfilador de memoria se concluye que la mayoria de la memoria reportada (13.9 MiB) es ocupada por la linea 13. Por misma razon que antes, esto corresponde al llamado de subreddit.hot(limit=n_hot), dado que recibe los resultados de la peticion realizada a la API. A diferencia de antes, el ciclo for no tiene una contribucion significativa en la memoria ocupada por la funcion.

### Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

**Respuesta:**

En resumen, tanto para el consumo de memoria como de tiempo es debido a la linea 13. Que corresponde a la peticion relizada a la API (tiempo) que devuelve los post mas "hot" (espacio). Tambien se menciono anteriormente que el ciclo for de la linea 18, consume otra gran parte del tiempo de ejecucion de la funcion. Lo utlimo, puede deberse a la gran cantidad de iteraciones que realizan las linea de codigo dentro del ciclo for (ver columna Hits en el reporte del perfilador de tiempo, o Ocurrences en el reporte del perfilador de memoria). Tambien notamos que la funcion tiene un try/except como medida de seguridad. Sin embargo, durante la ejecucion nunca se entra a la excepción.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

### Responder las preguntas

**Respuestas Teóricas:**

1.a) [Memoria cache] La memoria cache es un tipo de memoria que nos permite almacenar informacion (datos). Este tipo de memoria tiene el unico fin de acceder rapidamente en consultas posteriores. Dichas consultas (o accesos de memoria), tienen la caracteristica de ser repetitivas. Un ejemplo practico de su uso es en funciones recursivas, las cuales suelen ser optimizadas con programacion dinamica. 

1.b) [LRU] Son las siglas en ingles de "Menos usado recientemente" (Least recently used). Este es un algoritmo basico para gestionar la memoria cache. La memoria de este tipo suele tener un largo pequeño, equivalente al numero de elementos almacenados. Si la memoria cache esta lleba, y se desea añadir un elemenot en la memoria, tal como lo dice el nombre del algoritmo, se descarta el elemento menos usado recientemente en la memoria. Para esto, el algoritmo debe de mantener contadores para identificar los elementos menos usados (o consultados).

2.- Este tipo de algoritmos se utiliza para reducir el tiempo de ejecucion de una funcion (o codigo con inputs y output deterministico). La forma de lograr esto, es intercambiando tiempo por espacio. En este caso, el espacio es sinonimo de memoria. Tal como se motivo en la pregunta anterior, dicha memoria es de tipo cache. En efecto, los costos de utilizar técnicas de Caching son en memoria (o espacio).

3.- La consecuencia de ocupar caching en la función anterior es que al realizar otra vez el mismo llamado de esta funcion, se recibira el mismo output que quedo almacenado en memoria cache. Pese a que esto puede ser una buena idea, ya que el tiempo de ejecucion es considerable, utilizar este metodo no es del todo apropiado. Esto se debe a que la respuesta de la funcion varía con el tiempo. Como se reciben los hot-post, en un par se semanas hasta incluso dias, los post mas hot debiesen de ser otros.

### Mejorar el código con cache.

In [9]:
@lru_cache
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

### Explicar las consecuencias de usar lru_cache sobre praw_reddit

Sin Caché:

In [10]:
%%timeit
votes1, post1, url1 = praw_reddit()

The slowest run took 14.00 times longer than the fastest. This could mean that an intermediate result is being cached.
371 ns ± 423 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


Con caché - Primer llamado:

In [11]:
%%timeit
votes3, post3, url3 = praw_reddit()

100 ns ± 2.79 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Con caché - Segundo llamado:

In [12]:
%%timeit
votes4, post4, url4 = praw_reddit()

103 ns ± 4.59 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Comentarios: Se puede observar que la ejecución normal se demora notablemente, hasta 10 órdenes de magnitud más que la ejecución con caché. La diferencia entre la ejecución con y sin caché es principalmente que la ejecución con caché está guardando el resultado y no ejecuta los pasos intermedios al volver a llamarse.

La principal consecuencia de usar caching en este ya fue expuesta anteriormente, y es que al ejecutarse la función no está pidiendo información nueva al servidor, lo que estanca los resultados, pudiendo ser contraproducente con una función que pretende evaluar la actualidad de un tema en reddit.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [13]:
def mu_sgm_python(botes):
    out = []
    for (u, d) in botes:
        a, b = 1+u, 1+d
        sgm = 1.65 * ( ( a*b / ( (a+b)**2 * (a+b+1))) ** 0.5 )
        mu = a / (a+b)
        out.append([mu, sgm])
    return out

In [14]:
%%timeit
mu_sgm_python(votes[9])

1.71 ms ± 58.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
def mu_sgm_numpy(botes):
    a_b = botes+np.ones(1)
    a, b = a_b[:,0], a_b[:,1]
    sgm = 1.65 * np.sqrt( a*b / ((a+b)**2*(a+b+1)))
    mu = a/(a+b)
    out = np.column_stack((mu,sgm))
    return out

In [16]:
%%timeit
mu_sgm_numpy(votes[9])

32.3 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [17]:
@jit(nopython=True)
def mu_sgm_python_jit(botes):
    out = []
    for (u, d) in botes:
        a, b = 1+u, 1+d
        sgm = 1.65 * ( ( a*b / ( (a+b)**2 * (a+b+1))) ** 0.5 )
        mu = a / (a+b)
        out.append([mu, sgm])
    return out

In [18]:
%%timeit
mu_sgm_python_jit(votes[9])

159 µs ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
@jit(nopython=True)
def mu_sgm_numpy_jit(botes):
    a_b = botes+np.ones(1)
    a, b = a_b[:,0], a_b[:,1]
    sgm = 1.65 * np.sqrt( a*b / ((a+b)**2*(a+b+1)))
    mu = a/(a+b)
    out = np.column_stack((mu,sgm))
    return out

In [20]:
%%timeit
mu_sgm_numpy_jit(votes[9])

11.3 µs ± 7.66 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Calentar funciones:

In [21]:
_ = mu_sgm_python_jit(votes[9]), mu_sgm_numpy_jit(votes[9])

In [22]:
%%timeit
mu_sgm_python_jit(votes[9])

194 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%%timeit
mu_sgm_numpy_jit(votes[9])

9.51 µs ± 477 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
if __name__ == '__main__':
    import timeit
    data = []
    dic_fun = {
        "Python": "mu_sgm_python",
        "Numpy": "mu_sgm_numpy",
        "Python-JIT": "mu_sgm_python_jit",
        "Numpy-JIT": "mu_sgm_numpy_jit"
    }
    for k, v in dic_fun.items():
        for i in range(10):
            t = timeit.Timer(f"{v}(votes[{i}])", f"from __main__ import {v}, votes")
            t = t.timeit(number=1000)
            o = {"Función": k, "N° de datos": len(votes[i]), "Tiempo (ms)": t}
            data.append(o)

In [25]:
df_tabla = pd.DataFrame(data)
fig = px.line(df_tabla, x="N° de datos", y="Tiempo (ms)", color="Función",log_y=True)
fig.update_layout(
    title="Tiempo de ejecución de cada función c/r a la cantidades de datos"
)
fig.show()

**Comentarios:**

En primer lugar, les ejecuciones de Python, compiladas o no, se tardan en general más que las de Numpy. Esto se debe al ventajoso manejo de la memoria de Numpy, haciéndolo más eficiente que Python. Luego, la diferencia entre las compiladas y las no compiladas es notoria, dado que las funciones que fueron compiladas se demoran aproximadamente 1 orden de magnitud menos, ya sea Python o Numpy. Esto último se debe a que la compilación previa optimiza el código, al comunicarse de forma más eficiente con el procesador.

En Python, la diferencia de tiempos entre ejecución compilada y no compilada se mantiene para todas las cantidades de datos, aunque se incremente el tiempo de ejecución. En cambio, para Numpy, se puede observar que las diferencias de los tiempos compilado y no compilado se acrecenta hacia el lado derecho del gráfico. No logramos decifrar una razon de esto.

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [26]:
def intervals_numpy(x, y):
    o = mu_sgm_numpy_jit(np.column_stack((x,y)))
    return o[:,0], o[:,1] 

In [27]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post
df

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



Votos (+)  Votos (-)                                               Post  \
0         270          0   Juraba que esta wea era un sueño febril de chico   
1         237          0  Gobierno queda fuera de querella por asesinato...   
2         222          0  Empresa le paga por error sueldo de $ 165 mill...   
3         214          0  Macabro hallazgo en San José de Maipo: perro l...   
4        1041         11                                             fonasa   
5         717          7                                       ahora no po'   
6         775          8                                              Bolos   
7         526          5  la situación de los pudúes en Chiloé es crític...   
8         426          4                                      Otros tiempos   
9         474          5                           Con razón hay tanto taco   
10        379          4  Alumna fue apuñalada tras oponerse a continuid...   
11        308          3  Hoy falleció la profesora Victoria Castro, una...   
12        275          3          Toxicidad en el área de la salud en Chile   
13        210          2  Alcalde de Arica advierte alarmante situación ...   
14        207          2  Una curiosa nota periodística publicada en Sep...   
15        204          2  Médicos usaban RUT de pacientes Fonasa para si...   
16        854         17  Encontré una joya ctm. Japoneses del colegio c...   
17        850         17                              Los límites del humor   
18        196          2  Detienen a enfermera y su pareja por ataque co...   
19        755         15  Venezolano en Chile explica las razones por la...   

                                                  url  
0                 https://i.redd.it/zvyt2znkta891.jpg  
1   https://www.eldinamo.cl/pais/Gobierno-queda-fu...  
2   https://www.t13.cl/noticia/diario-financiero/n...  
3   https://www.biobiochile.cl/noticias/nacional/r...  
4                 https://i.redd.it/az7ge5c891691.png  
5                 https://i.redd.it/83odxdkaq9791.png  
6                 https://i.redd.it/8tx19xd0k2691.jpg  
7               https://www.reddit.com/gallery/vhblmb  
8                 https://i.redd.it/3dgqvverce791.jpg  
9                 https://i.redd.it/nf9c4wh63g791.jpg  
10  https://m.cooperativa.cl/noticias/pais/region-...  
11                https://i.redd.it/rj35h02hko791.jpg  
12  https://www.reddit.com/r/chile/comments/vnewi3...  
13  https://www.theclinic.cl/2022/06/17/alcalde-ar...  
14                https://i.redd.it/bdf8e3tty9791.png  
15  https://www.meganoticias.cl/nacional/379981-me...  
16                    https://v.redd.it/chwx46xj3a691  
17                https://i.redd.it/ypmeusphce691.jpg  
18  https://www.biobiochile.cl/noticias/nacional/r...  
19                    https://v.redd.it/pfwny8irk8691

In [28]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

###  ¿Qué logra observar de los resultados?, ¿La solución resulta trivial? En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

**Respuesta:**

De lo que se puede observar en el gráfico, la rutina ordena los posts utilizando el límite inferior del intervalo de confianza equivalente a $\mu \pm 1 \sigma$. Dado esto, se espera que, en la mayoría de los casos, las medias $\mu$ también se encuentren ordenadas de mayor a menor, pero no existe una restricción que controle que esto se cumpla necesariamente.

Ahora, observando los valores del dataframe, se puede concluir que el proceso de ranking no es trivial, ya que los posts que se encuentran arriba tienen menos "upvotes" que algunos posts más abajo en el ranking, provocando que, por ejemplo, un post de 200 upvotes esté arriba de uno de más de 1000. Esto se explica por el cálculo de $\mu$ y $\sigma$, ya que a estas escalas, una pequeña cantidad de downvotes puede afectar notablemente su ranking. Por ejemplo, existen posts con 200 uptvotes al top del ranking, pero al tener solo 2 downvotes ya bajan casi al final del ranking.

En conclusión, parece una buena medida de premiar a los posts con muchos votos positivos y pocos negativos, ya que la proporción hará que estén en el top, por ejemplo, 1000 votos positivos versus 10 negativos. Pero además, facilita que posts nuevos que les está yendo bien, como 200 upvotes y ninguno negativo, también estén en el ranking, previniendo así que los posts con más cantidad de votos se estanquen en el top. Esta característica mantiene fresca la oferta de posts que se muestran al usuario, pero es muy competitiva, ya que es muy sensible a  que los pocos downvotes que tenga algún post lo hagan bajar varios lugares.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>